## Fine tune a llama3 model with m-a-p/Code-Feedback

### Setup
Let's start by installing all the lib we need to do supervised fine-tuning. We're going to use

Transformers for the LLM which we're going to fine-tune
Datasets for loading a SFT dataset from the hub, and preparing it for the model
BitsandBytes and PEFT for fine-tuning the model on consumer hardware, leveraging Q-LoRa, a technique which drastically reduces the compute requirements for fine-tuning
TRL, a library which includes useful Trainer classes for LLM fine-tuning.

In [ ]:
#!pip install -q transformers[torch] datasets
#!pip install -q bitsandbytes trl peft
#!pip install flash-attn --no-build-isolation

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

### Load Data + Preprocessing
The dataset contains various splits, each with a certain number of rows. In our case, as we're going to do supervised fine-tuning (SFT), only the "train_sft" and "test_sft" splits are relevant for us.


In [2]:
from datasets import load_dataset, DatasetDict

In [3]:
raw_datasets = load_dataset("m-a-p/Code-Feedback")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'messages'],
        num_rows: 66383
    })
})

In [ ]:
# SPLIT data in train/test
indices_1 = range(0,1000)
indices_2 = range(1001,2001)
dataset_dict = {
    "train": raw_datasets["train"].select(indices_1),
    "test": raw_datasets["train"].select(indices_2)
}
raw_dataset = DatasetDict(dataset_dict)
raw_dataset

In [ ]:
raw_dataset["train"][0]

In [4]:
# load data from s3 bucket
import boto3
import sagemaker

region_name = 'eu-central-1'

session = boto3.Session(region_name=region_name)
s3_sess = session.client('s3')
sm_session = sagemaker.Session(boto_session=session)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [5]:
training_input_path = f's3://{sm_session.default_bucket()}/datasets/answer-gen-coherence/train_dataset.json'
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

data

{'training': 's3://sagemaker-eu-central-1-505049265445/datasets/answer-gen-coherence/train_dataset.json'}

In [6]:
raw_dataset = load_dataset(
        "json",
        data_files=data
    )

raw_dataset

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


DatasetDict({
    training: Dataset({
        features: ['messages'],
        num_rows: 1269
    })
})

In [7]:
indices_1 = range(0,1000)
indices_2 = range(1001,1269)
dataset_dict = {
    "train": raw_dataset["training"].select(indices_1),
    "test": raw_dataset["training"].select(indices_2)
}
raw_dataset = DatasetDict(dataset_dict)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 268
    })
})

#### Tokenizer
Next, we instantiate the tokenizer, which is required to prepare the text for the model. The model doesn't directly take strings as input, but rather input_ids, which represent integer indices in the vocabulary of a Transformer model. 

We also set some attributes which the tokenizer of a base model typically doesn't have set, such as:

- the padding token ID. During pre-training, one doesn't need to pad since one just creates blocks of text to predict the next token, but during fine-tuning, we will need to pad the (instruction, completion) pairs in order to create batches of equal length.
- the model max length: this is required in order to truncate sequences which are too long for the model. Here we decide to train on at most 2048 tokens.
- the chat template. A chat template determines how each list of messages is turned into a tokenizable string, by adding special strings in between such as <|user|> to indicate a user message and <|assistant|> to indicate the chatbot's response. Here we define the default chat template, used by most chat models. See also the docs.

In [8]:
from transformers import AutoTokenizer
from huggingface_hub import login

In [9]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [10]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [11]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

#### Apply chat template
Once we have equipped the tokenizer with the appropriate attributes, it's time to apply the chat template to each list of messages. Here we basically turn each list of (instruction, completion) messages into a tokenizable string for the model.

Note that we specify tokenize=False here, since the SFTTrainer which we'll define later on will perform the tokenization internally. Here we only turn the list of messages into strings with the same format.

In [12]:
import re
import random
from multiprocessing import cpu_count


def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

In [13]:
column_names = list(raw_dataset["train"].features)
column_names


['messages']

In [14]:
# applies the apply_chat_template function to each element in raw_dataset using multiple CPU cores, passing a tokenizer and removing specified columns, with a progress description.
raw_dataset = raw_dataset.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template")

In [15]:
train_data = raw_dataset["train"]
test_data = raw_dataset["test"]

for index in random.sample(range(len(raw_dataset["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_dataset['train'][index]['text']}")
  

Sample 199 of the processed training set:

<|system|>
You are an student learning English. You have to answer the prompt with a level of coherence from 1 to 6 based on your ability to structure and connect ideas.
The coherence criteria range from 1 to 6, with each level representing increasing proficiency in coherence and organization.

Here is the scale you should use to build your answer:

1: Your answer lacks coherence: able to make very basic connections between words using basic conjunctions such as 'and'.

2: Your answer has limited coherence: able to make connections between short phrases using basic conjunctions such as 'because' and 'but'.

3: Your answer has moderate coherence: able to connect short phrases into a sequential series of ideas or events.

4: Your answer is fairly coherent: logically structures and organizes utterances and connects ideas using a limited number of discourse markers and conjunctions. A lack of organization or logical connection may be evident in lo

### Model Definition

With regular LoRa, one would keep the base model in 32 or 16 bits in memory, and then train the parameter weights. However, there have been new methods developed to shrink the size of a model considerably, to 8 or 4 bits per parameter (we call this "quantization"). Hence, if we apply LoRa to a quantized model (like a 4-bit model), then we call this QLoRa. We have a blog post that tells you all about it. There are various quantization methods available, here we're going to use the BitsandBytes integration.



In [16]:
# QloRa SFT
from transformers import BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load the model with 4-bit precision to reduce memory usage.
    bnb_4bit_quant_type="nf4",  # Use the "nf4" quantization type for 4-bit precision.
    bnb_4bit_compute_dtype=torch.bfloat16  # Perform computations using the bfloat16 data type.
)

device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None
device_map


{'': 0}

In [17]:
model_kwargs = dict(
    #attnt_implementation=True,  # (Commented out) Use a specific attention implementation.
    torch_dtype="auto",  # Automatically choose the appropriate torch data type (e.g., float32, float16).
    use_cache=False,  # Disable caching to save memory since gradient checkpointing is used.
    device_map=device_map,  # Map model layers to specific devices (e.g., GPUs).
    quantization_config=quant_config.to_dict()  # Apply model quantization settings by converting them to a dictionary.
)

### SFT Trainer

Next, we define the SFTTrainer available in the TRL library. This class inherits from the Trainer class available in the Transformers library, but is specifically optimized for supervised fine-tuning (instruction tuning). It can be used to train out-of-the-box on one or more GPUs, using Accelerate as backend.

Most notably, it supports packing, where multiple short examples are packed in the same input sequence to increase training efficiency.

As we're going to use QLoRa, the PEFT library provides a handy LoraConfig which defines on which layers of the base model to apply the adapters. One typically applies LoRa on the linear projection matrices of the attention layers of a Transformer. We then provide this configuration to the SFTTrainer class. The weights of the base model will be loaded as we specify the model_id (this requires some time).

We also specify various hyperparameters regarding training, such as:

- we're going to fine-tune for 1 epoch
- the learning rate and its scheduler
- we're going to use gradient checkpointing (yet another way to save memory during training)
- and so on.

In [18]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers import TrainingArguments

In [19]:
output_dir = "../../model_saved/llama_coherence_answer-gen-8B"

In [20]:
# based on config
training_args = TrainingArguments(
    fp16=True,  # Use 16-bit floating point precision (faster and less memory).
    do_eval=True,  # Perform evaluation during training.
    eval_strategy="epoch",  # Evaluate the model at the end of each training epoch.
    gradient_accumulation_steps=128,  # Accumulate gradients over 128 steps before updating weights.
    gradient_checkpointing=True,  # Save memory by not storing intermediate activations.
    gradient_checkpointing_kwargs={"use_reentrant": False},  # Disable reentrant autograd for gradient checkpointing.
    learning_rate=2.0e-05,  # Set the learning rate for the optimizer.
    log_level="info",  # Set the level of detail for logging information.
    logging_steps=5,  # Log information every 5 steps.
    logging_strategy="steps",  # Log based on steps, not epochs.
    lr_scheduler_type="cosine",  # Use a cosine schedule to adjust the learning rate.
    max_steps=-1,  # Train indefinitely unless a stopping criterion is met.
    num_train_epochs=1,  # Train for 1 full epoch.
    output_dir=output_dir,  # Directory to save training outputs and checkpoints.
    overwrite_output_dir=True,  # Overwrite the output directory if it exists.
    per_device_eval_batch_size=1,  # Use a batch size of 1 per device for evaluation (previously set to 8).
    per_device_train_batch_size=1,  # Use a batch size of 1 per device for training (previously set to 8).
    save_strategy="no",  # Don't save model checkpoints during training.
    save_total_limit=None,  # No limit on the number of checkpoints saved.
    seed=42,  # Set the random seed for reproducibility.
)

# based on config
peft_config = LoraConfig(
    r=64,  # Set the rank for the LoRA (low-rank adaptation) matrices.
    lora_alpha=16,  # Scaling factor for the LoRA updates.
    lora_dropout=0.1,  # Apply dropout with a probability of 0.1 to LoRA layers.
    bias="none",  # No additional bias terms are added in the LoRA layers.
    task_type="CAUSAL_LM",  # Specify the task type as causal language modeling.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Apply LoRA to these specific model modules.
)




In [21]:
trainer = SFTTrainer(
    model=model_id,  # Use the specified model ID for training.
    model_init_kwargs=model_kwargs,  # Pass in model initialization arguments (like device mapping and dtype).
    args=training_args,  # Apply the defined training arguments (e.g., learning rate, batch size).
    train_dataset=train_data,  # Use the provided dataset for training.
    eval_dataset=test_data,  # Use the provided dataset for evaluation.
    dataset_text_field="text",  # Specify the field containing text data in the datasets.
    tokenizer=tokenizer,  # Use the specified tokenizer for preprocessing the text data.
    packing=True,  # Enable sequence packing to maximize training efficiency.
    peft_config=peft_config,  # Apply the specified parameter-efficient fine-tuning (PEFT) configuration.
    max_seq_length=tokenizer.model_max_length,  # Set the maximum sequence length for the model based on the tokenizer's max length.
)


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': model_init_kwargs, dataset_text_field, packing, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:155: UserWarning: You passed `model_init_kwargs` to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:185: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(
Unused kwargs: ['quant_method'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend


In [22]:
train_result = trainer.train()

***** Running training *****
  Num examples = 168
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 128
  Total optimization steps = 1
  Number of trainable parameters = 54,525,952


Epoch,Training Loss,Validation Loss



***** Running Evaluation *****
  Num examples = 45
  Batch size = 1
  Num examples = 45
  Batch size = 1


### Saving Model

In [23]:
metrics = train_result.metrics
max_train_samples = 100
metrics["train_samples"] = min(max_train_samples, len(train_data))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
trainer.save_model(training_args.output_dir)

Saving model checkpoint to ../../model_saved/llama_coherence_answer-gen-8B


***** train metrics *****
  epoch                    =     0.7619
  total_flos               = 11073414GF
  train_loss               =     0.9628
  train_runtime            = 0:06:46.53
  train_samples            =        100
  train_samples_per_second =      0.413
  train_steps_per_second   =      0.002


loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6/config.json
Model config LlamaConfig {
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 128256
}

tokenizer config file saved in ../../model_saved/llama_coherence_answer-gen-8B/tokenizer_config.json
Special token

### Inference

In [25]:
from transformers import AutoTokenizer, AutoModelForCausalLM

output_dir = "../../model_saved/llama_coherence_answer-gen-8B"

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_sc

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file generation_config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "max_length": 4096,
  "temperature": 0.6,
  "top_p": 0.9
}



In [28]:
import torch

# We use the tokenizer's chat template to format each message
messages = [
    {"role": "user", "content": "Answer a prompt like What did you do during your last vacation with a level of coherence 2."},
]

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

# Prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# Ensure input_ids are valid
print("Input IDs:", input_ids)

# Inference with model output checks
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=1.0,  # Adjusted for stability
        top_k=2,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True
)

# Decode the generated output
decoded_output = tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=True)[0]
print(decoded_output)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Input IDs: tensor([[   198,  16533,    264,  10137,   1093,   3639,   1550,    499,    656,
           2391,    701,   1566,  20769,    449,    264,   2237,    315,  78925,
            220,     17,     13, 128001,    271]], device='cuda:0')

Answer a prompt like What did you do during your last vacation with a level of coherence 2.

Answer a prompt like What did you do during your last vacation with a level of coherence 2. respond
Answer a prompt like What did you do during your last vacation with a level of coherence 2. respond

Answer a prompt like What did you do during your last vacation with a level of coherence 2. respond
Answer a prompt like What did you do during your last vacation with a level of coherence 2. respond

Answer a prompt like What did you do during your last vacation with a level of coherence 2. respond
Answer a prompt like What did you do during your last vacation with a level of coherence 2. respond

Answer a prompt like What did you do during your last vacation